In [1]:
import pandas as pd
import numpy as np
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianNetwork
import scipy.stats

import sklearn

In [2]:
df = pd.read_csv("patients_info.csv")

In [3]:
is_null_feature_df = df.isnull().any()
is_null_indexs = is_null_feature_df.loc[is_null_feature_df == True].index
print(df[is_null_indexs].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Empty DataFrameNone


In [4]:
df2 = df.dropna(axis=1, how='any')
df.columns

Index(['Unnamed: 0', 'subject_id', 'gender', 'dob', 'dod_hosp', 'dod_ssn',
       'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'has_chartevents_data', 'submit_wardid',
       'curr_wardid', 'curr_careunit', 'callout_wardid', 'callout_service',
       'request_tele', 'request_resp', 'request_cdiff', 'request_mrsa',
       'request_vre', 'callout_status', 'callout_outcome', 'discharge_wardid',
       'acknowledge_status', 'createtime', 'updatetime', 'acknowledgetime',
       'outcometime'],
      dtype='object')

In [5]:
from numpy import NaN

# df2.loc[:, ['SUBJECT_ID']] = NaN
df2

,Unnamed: 0,subject_id,gender,dob,dod_hosp,dod_ssn,hadm_id,admittime,dischtime,deathtime,...,request_mrsa,request_vre,callout_status,callout_outcome,discharge_wardid,acknowledge_status,createtime,updatetime,acknowledgetime,outcometime
0,0,48637,F,2044-10-27,2200-01-01,2200-01-01,148117,2133-06-08 17:02:00,2133-06-16 17:20:00,2200-01-01 00:00:00,...,0,0,Inactive,Cancelled,100.0,Acknowledged,2133-06-10 14:30:46,2133-06-12 12:43:10,2133-06-12 12:39:40,2133-06-12 12:43:10
1,1,48637,F,2044-10-27,2200-01-01,2200-01-01,148117,2133-06-08 17:02:00,2133-06-16 17:20:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,31.0,Acknowledged,2133-06-12 19:22:40,2133-06-12 19:51:48,2133-06-12 19:52:24,2133-06-12 21:55:24
2,2,9216,M,2170-04-19,2200-01-01,2200-01-01,125531,2198-05-26 15:37:00,2198-06-01 17:30:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,18.0,Acknowledged,2198-05-31 10:18:12,2198-05-31 10:18:12,2198-05-31 10:27:09,2198-05-31 23:25:02
3,3,9216,M,2170-04-19,2200-01-01,2200-01-01,178939,2198-11-11 20:14:00,2198-11-16 18:47:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,45.0,Acknowledged,2198-11-15 08:55:48,2198-11-15 08:55:48,2198-11-15 09:05:31,2198-11-15 19:10:02
4,4,9216,M,2170-04-19,2200-01-01,2200-01-01,107657,2199-04-05 20:32:00,2199-04-11 12:00:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,18.0,Acknowledged,2199-04-10 12:04:42,2199-04-10 12:04:42,2199-04-10 12:04:54,2199-04-10 17:25:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,987,11133,F,2103-10-30,2200-01-01,2200-01-01,184664,2174-01-11 21:17:00,2174-01-21 13:43:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,40.0,Acknowledged,2174-01-14 07:45:12,2174-01-14 07:45:12,2174-01-14 07:46:15,2174-01-14 17:55:02
988,988,79878,M,2093-10-14,2200-01-01,2200-01-01,154650,2156-12-24 14:51:00,2156-12-27 20:20:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,17.0,Acknowledged,2156-12-26 08:21:55,2156-12-26 13:00:56,2156-12-26 13:00:58,2156-12-26 15:10:22
989,989,64145,F,2114-07-25,2200-01-01,2200-01-01,109844,2194-02-22 18:11:00,2194-03-07 15:17:00,2200-01-01 00:00:00,...,0,0,Inactive,Cancelled,100.0,Acknowledged,2194-03-01 14:27:29,2194-03-01 17:30:32,2194-03-01 14:37:49,2194-03-01 17:30:32
990,990,64145,F,2114-07-25,2200-01-01,2200-01-01,109844,2194-02-22 18:11:00,2194-03-07 15:17:00,2200-01-01 00:00:00,...,0,0,Inactive,Discharged,17.0,Acknowledged,2194-03-03 09:13:40,2194-03-03 09:13:40,2194-03-03 09:22:54,2194-03-03 19:25:08


In [6]:
from pgmpy.estimators import HillClimbSearch

hc = HillClimbSearch(df2)
best_model = hc.estimate()
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [ ]:
my_model = BayesianNetwork(best_model.edges())

In [ ]:
my_model.fit(df2, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in my_model.get_cpds():
    print(cpd)

+-------------------+-----+-----------------------------+
| INTIME            | ... | INTIME(2161-03-14 20:40:24) |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(376)   | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(4838)  | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(11945) | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(22192) | ... | 0.005780346820809248        |
+-------------------+-----+-----------------------------+
| SUBJECT_ID(32443) | ... | 0.976878612716763           |
+-------------------+-----+-----------------------------+
+-----------------+-----------------+-----+----------------------+
| SUBJECT_ID      | SUBJECT_ID(376) | ... | SUBJECT_ID(32443)    |
+-----------------+-----------------+-----+----------------------+
| DOB(1828-07-20) | 0.025           | ... | 0

In [ ]:
samples = BayesianModelSampling(my_model).forward_sample(size=int(1e5))

  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
samples.loc[(samples['SUBJECT_ID'] == 11945), :]['EVENTTYPE']

3         transfer
4            admit
6         transfer
9         transfer
12        transfer
           ...    
99974     transfer
99976     transfer
99979        admit
99991     transfer
99992    discharge
Name: EVENTTYPE, Length: 29754, dtype: object

In [ ]:
df2.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'GENDER', 'DOB', 'EXPIRE_FLAG', 'HADM_ID',
       'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS',
       'ETHNICITY', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'ICUSTAY_ID', 'DBSOURCE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME',
       'LOS', 'TRANSFERTIME', 'CURR_SERVICE', 'EVENTTYPE'],
      dtype='object')

In [ ]:
for i in df2.columns:
    if i not in samples.columns:
        print(i)

ADMISSION_TYPE
HAS_CHARTEVENTS_DATA


In [ ]:
KL = scipy.stats.entropy(samples, df2)

TypeError: can't multiply sequence by non-int of type 'float'

---

In [1]:
from pgmpy.estimators import HillClimbSearch
import pandas as pd
import numpy as np

In [28]:
# create some data with dependencies
data = pd.DataFrame(np.random.randint(0, 3, size=(25000, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']
data['D'] = data['H'] - data['A']
data['F'] = data['H'] - data['D']

hc = HillClimbSearch(data)
best_model = hc.estimate()
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('A', 'F'), ('A', 'B'), ('A', 'C'), ('C', 'B'), ('D', 'A'), ('D', 'G'), ('G', 'A'), ('H', 'D'), ('H', 'G')]


In [30]:
model = BayesianNetwork(best_model.edges())

In [36]:
model.edges

OutEdgeView([('A', 'F'), ('A', 'B'), ('A', 'C'), ('C', 'B'), ('D', 'A'), ('D', 'G'), ('G', 'A'), ('H', 'D'), ('H', 'G')])

In [33]:
model.fit(data)

In [34]:
model.fit_update(data.loc[:, list(model.nodes)])

In [35]:
model.save(filename="test_save_model.bif", filetype='bif')

In [4]:
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
model = get_example_model('alarm')
# Generate some new data.
data = BayesianModelSampling(model).forward_sample(int(100))
model.fit_update(data)

  0%|          | 0/37 [00:00<?, ?it/s]

In [6]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD

student = BayesianNetwork([('diff', 'grades'), ('intel', 'grades')])

In [7]:
grades_cpd = TabularCPD('grades', 3, [[0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.1,0.1,0.1,0.1,0.1,0.1],
                                      [0.8,0.8,0.8,0.8,0.8,0.8]],
                        evidence=['diff', 'intel'], evidence_card=[2, 3])
student.add_cpds(grades_cpd)

In [20]:
student.simulate

pgmpy.models.BayesianNetwork.BayesianNetwork

In [21]:
student.save(filename="test222.bif", filetype='bif')

KeyError: 'diff'